<a href="https://colab.research.google.com/github/jmlDC/MediaBias-Thesis22-23/blob/Modeling/Cleaner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mounting Gdrive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

dir  = "/content/gdrive/MyDrive/THESIS-MS/Git-Thesis22-23/"

Mounted at /content/gdrive


## Imports

In [10]:
import pandas as pd
import nltk
nltk.download('punkt')

from nltk.tokenize import sent_tokenize


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 31.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=21b0c2a90a97e47fe53596102ae84a218269208f6646c66861f406477cd0b3b3
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


### Function

In [ ]:
# Stop words and empty lines
# (PNA photo by
# (Photo courtesy by ) not captured
import re

def make_annotateDF(df, isSunStar=False):
    workingDF = pd.DataFrame(columns=['Annotate','Text'])

    for index, row in df.iterrows():
        # print(df['Title'].iloc[index])
        if (row['Title'] == 'MB Daily News Summary'):
            pass
        else:
            new_row = pd.DataFrame({'Annotate':'', 'Text': df['Title'].iloc[index]}, index=[0])
            workingDF = pd.concat([workingDF.loc[:], new_row]).reset_index(drop=True)
            text = str(df['Text'].iloc[index])
            # print(text)
            if isSunStar:
                sentences = sent_tokenize(text)
            else:
                sentences = text.split('\n')
            for sentence in sentences:
                sentence = re.sub(r"http\S+", "", sentence)
                sentence = re.sub(r"@\S+", "", sentence)
                sentence = re.sub(r"\S+.com\S+", "", sentence)
                sentence = sentence.replace("– Rappler.com","")
                sentence = sentence.replace("(PNA)","")

                if (', GMA Integrated News' in sentence
                    or sentence ==''
                    or sentence =='– Rappler.com'
                    or 'SunStar Philippines)' in sentence
                    or 'File photo' in sentence
                    or 'Courtesy of ' in sentence
                    or len(sentence.split()) <3
                    or 'RELATED STORY' in sentence
                    or '(Photo: '  in sentence
                    or 'Rappler.com' in sentence
                    or '(Photo courtesy of' in sentence
                    ):
                    # print(sentence)
                    pass
                else:
                    new_row = pd.DataFrame({'Annotate':'', 'Text': sentence}, index=[0])
                    workingDF = pd.concat([workingDF.loc[:], new_row]).reset_index(drop=True)

    return workingDF


In [12]:
from langdetect import detect
import re

from langdetect import DetectorFactory
DetectorFactory.seed = 0

def get_headlineLead(df, isSunStar=False, isPNA=False):
    workingDF = pd.DataFrame(columns=['Text', 'Annotate'])

    for index, row in df.iterrows():
        # print(df['Title'].iloc[index])
        try:
            lang = detect(df['Title'].iloc[index])
        except:
            lang = ""

        if (lang == 'tl'):
            # print(df['Title'].iloc[index])
            pass
        if (row['Title'] == 'MB Daily News Summary'):
            pass
        else:
            new_row = pd.DataFrame({'Annotate':'', 'Text': df['Title'].iloc[index]}, index=[0])
            workingDF = pd.concat([workingDF.loc[:], new_row]).reset_index(drop=True)
            text = str(df['Text'].iloc[index])

            if isSunStar:
                text = re.sub(r'\.(?=[^ \W\d])', '. ', text)
                sentences = sent_tokenize(text)

            else:
                sentences = text.split('\n')

            if isPNA:
                for text in sentences:
                    if text.startswith('MANILA'):
                        sentence = text
                        sentence = sentence.strip()
                        break
            else:
                sentence = sentences[0].strip()



            sentence = re.sub(r"http\S+", "", sentence)
            sentence = re.sub(r"@\S+", "", sentence)
            sentence = re.sub(r"\S+.com\S+", "", sentence)
            sentence = sentence.replace("– Rappler.com","")
            sentence = sentence.replace("(PNA)","")

            if (', GMA Integrated News' in sentence
                    or sentence ==''
                    or sentence =='– Rappler.com'
                    or 'SunStar Philippines)' in sentence
                    or 'File photo' in sentence
                    or 'Courtesy of ' in sentence
                    or len(sentence.split()) <3
                    or 'RELATED STORY' in sentence
                    or '(Photo: '  in sentence
                    or 'Rappler.com' in sentence
                    or '(Photo courtesy of' in sentence
                    ):
                    # print(sentence)
                pass
            else:
                new_row = pd.DataFrame({'Annotate':'', 'Text': sentence}, index=[0])
                workingDF = pd.concat([workingDF.loc[:], new_row]).reset_index(drop=True)

    return workingDF


## Main

### GMA

In [13]:
gma = pd.read_csv(f'{dir}Official/GMA_2023-05-07NEWS.csv')
gma

,Unnamed: 0,Title,Author,Date,Text
0,0,"DOH logs 1,920 new COVID-19 cases, highest dai...",By GMA Integrated News,2023-05-07,"The Philippines on Sunday registered 1,920 ne..."
1,1,"DOH logs 9 new COVID-19 deaths; 1,878 fresh ca...",NaN,2023-05-07,The Department of Health said Saturday that t...
2,2,Bettor wins P55.3 million jackpot in Grand Lot...,By GMA Integrated News,2023-05-07,"A bettor won the jackpot prize of P55,394,255..."
3,3,Filipinos watch as King Charles III of Britain...,By GMA Integrated News,2023-05-06,Filipinos monitored the coronation of the Uni...
4,4,"Marcos will not abolish NTF-ELCAC, says NSC",By GMA Integrated News,2023-05-06,President Ferdinand R. Marcos Jr. has no inte...
...,...,...,...,...,...
2218,2218,"SC upholds House journal over videos, photos; ...","By JOAHNA LEI CASILAO, GMA Integrated News",2023-01-27,The Supreme Court has upheld the constitution...
2219,2219,Philippines' justice chief says ICC drug war p...,"By JOAHNA LEI CASILAO, GMA Integrated News",2023-01-27,Justice Secretary Jesus Crispin Remulla descr...
2220,2220,De Lima drug cases may be resolved by March —l...,"By JOAHNA LEI CASILAO, GMA Integrated News",2023-01-27,The camp of detained former Senator Leila de ...
2221,2221,Philippines to appeal ICC’s resumption of drug...,"By JOAHNA LEI CASILAO, GMA Integrated News",2023-01-27,The Philippine government intends to appeal t...


In [ ]:
gma_df = make_annotateDF(gma)
gma_df

,Annotate,Text
0,,"DOH logs 1,920 new COVID-19 cases, highest dai..."
1,,"The Philippines on Sunday registered 1,920 ne..."
2,,"According to GMA Integrated News Research, th..."
3,,"It is the fourth straight day of more than 1,..."
4,,Data from the Department of Health's COVID-19...
...,...,...
27781,,"A total of 1,964,679 individuals or 481,012 f..."
27782,,"Of the affected population, 88,134 individual..."
27783,,"A total of 1,858 houses were reported damaged..."
27784,,"Damage to agriculture worth PP885,165,517 and..."


In [14]:
gmaHL_df = get_headlineLead(gma)
gmaHL_df

,Text,Annotate
0,"DOH logs 1,920 new COVID-19 cases, highest dai...",
1,"The Philippines on Sunday registered 1,920 new...",
2,"DOH logs 9 new COVID-19 deaths; 1,878 fresh ca...",
3,The Department of Health said Saturday that th...,
4,Bettor wins P55.3 million jackpot in Grand Lot...,
...,...,...
4406,The camp of detained former Senator Leila de L...,
4407,Philippines to appeal ICC’s resumption of drug...,
4408,The Philippine government intends to appeal th...,
4409,Reported death toll from bad weather now at 40...,


### PNA

In [15]:
pna = pd.read_csv(f'{dir}Official/PhilippineNewsAgency_2023-05-06NEWS.csv')
pna

,Unnamed: 0,Title,Author,Date,Text
0,0,"MIAA, Meralco begins NAIA T3 electrical audit",NaN,"May 5, 2023",File photo\nMANILA – The Manila International ...
1,1,"Pag-IBIG, Singaporean firm eye building ‘net-z...",NaN,"May 5, 2023","MANILA – Marilene Acosta, the CEO of Pag-IBIG ..."
2,2,DMW assures aid for OFWS from Sudan who wanted...,By Marita Moaje,"May 5, 2023",HOME. A total of 74 repatriates from war-torn ...
3,3,PNP chief fetes HPG's vital role in upholding ...,By Christopher Lloyd Caliwan,"May 5, 2023",PNP chief Gen. Benjamin Acorda Jr. (PNA photo ...
4,4,"LTO chief: 'Fixers,' fake docs jeopardize publ...",By Raymond Carl Dela Cruz,"May 5, 2023",MANILA – The chief of the Land Transportation ...
...,...,...,...,...,...
2195,2195,PBBM's 'Pambansang Pabahay’ empowers LGUs to a...,By Zaldy De Layola,"February 8, 2023",(File photo)\nMANILA – The Department of Human...
2196,2196,1.2K cops to secure ASEAN education execs' mee...,By Christopher Lloyd Caliwan,"February 8, 2023",(Photo courtesy of NCRPO)\nMANILA – More than ...
2197,2197,PNP reaffirms support to Interpol efforts vs. ...,By Christopher Lloyd Caliwan,"February 8, 2023",Interpol Secretary General Jürgen Stock (left)...
2198,2198,"IP leaders call for unity to preserve IPRA, NC...","By Perfecto Raymundo, Jr. and Che Palicte","February 8, 2023",UNITY. Indigenous people leaders call for unit...


In [ ]:
# pna_df = make_annotateDF(pna)
# pna_df

,Annotate,Text
0,,"MIAA, Meralco begins NAIA T3 electrical audit"
1,,MANILA – The Manila International Airport Auth...
2,,MIAA officer-in-charge Bryan Co said the audit...
3,,"Co added that once the audit is concluded, the..."
4,,The MIAA will then determine the timeline as w...
...,...,...
28692,,He added that Philippine Rear Admiral Renato ...
28693,,"""The meeting will yield meaningful discussions..."
28694,,He added the staff-to-staff talks between the ...
28695,,Badidles added that the activity is part of th...


In [16]:
pnaHL_df = get_headlineLead(pna, isPNA=True)
pnaHL_df

,Text,Annotate
0,"MIAA, Meralco begins NAIA T3 electrical audit",
1,MANILA – The Manila International Airport Auth...,
2,"Pag-IBIG, Singaporean firm eye building ‘net-z...",
3,"MANILA – Marilene Acosta, the CEO of Pag-IBIG ...",
4,DMW assures aid for OFWS from Sudan who wanted...,
...,...,...
4395,"MANILA – More than 1,200 police officers have ...",
4396,"IP leaders call for unity to preserve IPRA, NC...",
4397,MANILA – Leaders of the indigenous peoples (IP...,
4398,"PH, US Navy fleets to tackle cooperation, exch...",


### Rappler

In [17]:
rappler = pd.read_csv(f'{dir}Official/Rappler_2023-05-07NEWS.csv')
rappler

,Unnamed: 0,Title,Author,Date,Text
0,0,Marcos the party-going schoolboy returns to UK...,Lian Buan,2023-05-05,"LONDON, United Kingdom – In the late ’70s, Fer..."
1,1,"Ablan confirms resignation from DepEd, denies ...",Ryan Macasero,2023-05-05,"MANILA, Philippines – Former Department of Edu..."
2,2,WATCH: Paano naibenta sa labas ang DepEd lapto...,Rappler.com,2023-05-05,"MANILA, Philippines – Naaalala ‘nyo ba ‘yung m..."
3,3,"In unprecedented move, Filipino journalist har...",Gaby Baizas,2023-05-05,"MANILA, Philippines – Days after media workers..."
4,4,Marcos says Philippines bases could be ‘useful...,Reuters,2023-05-05,"WASHINGTON DC, USA – Philippines President Fer..."
...,...,...,...,...,...
2196,2196,Fewer Filipinos skeptical of COVID-19 jabs – SWS,Sofia Tomacruz,2022-01-21,"MANILA, Philippines – As the coronavirus is po..."
2197,2197,"Duterte appoints Malacañang official, ex-Napol...",NaN,2022-01-21,"MANILA, Philippines – President Rodrigo Dutert..."
2198,2198,‘Ironclad’ PH-South Korea alliance seen with s...,Jairo Bolledo,2022-01-21,"MANILA, Philippines – The Philippines is looki..."
2199,2199,"3 Cordillera provinces, Northern Samar placed ...",NaN,2022-01-20,"MANILA, Philippines – The alert level of three..."


In [ ]:
rappler_df = make_annotateDF(rappler)
rappler_df

,Annotate,Text
0,,Marcos the party-going schoolboy returns to UK...
1,,"LONDON, United Kingdom – In the late ’70s, Fer..."
2,,"Marcos, his first lady Louise Araneta Marcos a..."
3,,"“Sa palagay ko, dahil magkakilala kami, eh dap..."
4,,"Besides, said Locsin, “he left his heart here.”"
...,...,...
33998,,"On January 22, Jimenez acknowledged that the T..."
33999,,“Because the ballots have not yet been printed...
34000,,The printing of ballots was supposed to start ...
34001,,Some 67.4 million ballots will be printed over...


In [18]:
rapplerHL_df = get_headlineLead(rappler)
rapplerHL_df

,Text,Annotate
0,Marcos the party-going schoolboy returns to UK...,
1,"LONDON, United Kingdom – In the late ’70s, Fer...",
2,"Ablan confirms resignation from DepEd, denies ...",
3,"MANILA, Philippines – Former Department of Edu...",
4,WATCH: Paano naibenta sa labas ang DepEd lapto...,
...,...,...
4394,"MANILA, Philippines – The Philippines is looki...",
4395,"3 Cordillera provinces, Northern Samar placed ...",
4396,"MANILA, Philippines – The alert level of three...",
4397,Comelec starts printing ballots for 2022 polls,


### SunStar

In [19]:
sunstar = pd.read_csv(f'{dir}Official/SunStar_2023-05-07NEWS.csv')
sunstar

,Unnamed: 0,Title,Author,Date,Text
0,0,Sandiganbayan affirms denial of motion seeking...,Third Anne Peralta-Malonzo,"May 07, 2023",THE Sandiganbayan has affirmed the decision of...
1,1,PH to reassess protocols with end of Covid glo...,NaN,"May 06, 2023",WITH the end of the global emergency status fo...
2,2,DMW: Work experience top criterion for oversea...,NaN,"May 06, 2023",THE Department of Migrant Workers (DMW) on Sat...
3,3,Marcos accepts courtesy resignation of 2 senio...,Third Anne Peralta-Malonzo,"May 05, 2023",PRESIDENT Ferdinand “Bongbong” Marcos has conf...
4,4,"DOH reiterates call for kids to get measles, p...",NaN,"May 05, 2023",FIVE days after starting its mass immunization...
...,...,...,...,...,...
2261,2261,"CSF indigents get cash aid, food packs from Guv",Ian Ocampo Flora,"January 30, 2023",CITY OF SAN FERNANDO The provincial government...
2262,2262,AC execs help Bale Pusu ward reunite with family,Charlene A. Cayabyab,"January 28, 2023",ANGELES CITY The city government here has faci...
2263,2263,Cops seize P14-M 'shabu' in Bulacan,Princess Clea Arcellaz,"January 28, 2023",CITY OF SAN FERNANDO More than two kilos of su...
2264,2264,Crissot's 156 birth anniversary commemorated,Ian Ocampo Flora,"January 28, 2023",BACOLOR The heirs of Juan Crisostomo Soto (Cri...


In [ ]:
sunstar_df = make_annotateDF(sunstar, isSunStar=True)
sunstar_df

,Annotate,Text
0,,Sandiganbayan affirms denial of motion seeking...
1,,THE Sandiganbayan has affirmed the decision of...
2,,A careful reading of the defendants’ motion sh...
3,,"In view of the foregoing, the defendants faile..."
4,,The Court thus maintains its earlier pronounce...
...,...,...
17797,,The said session was attended by representativ...
17798,,"During the inquiry, SFELAPCO representatives e..."
17799,,It was also disclosed that the expiration of S...
17800,,Macalino assailed SFELAPCO’s alleged lack of c...


In [20]:
sunstarHL_df = get_headlineLead(sunstar, isSunStar=True)
sunstarHL_df

,Text,Annotate
0,Sandiganbayan affirms denial of motion seeking...,
1,THE Sandiganbayan has affirmed the decision of...,
2,PH to reassess protocols with end of Covid glo...,
3,WITH the end of the global emergency status fo...,
4,DMW: Work experience top criterion for oversea...,
...,...,...
4516,CITY OF SAN FERNANDO More than two kilos of su...,
4517,Crissot's 156 birth anniversary commemorated,
4518,BACOLOR The heirs of Juan Crisostomo Soto (Cri...,
4519,CSF council inquiry on power rate hike launched,


### Manila Bulletin

In [21]:
mb = pd.read_csv(f'{dir}Official/manilaBulletin_2023-05-05_NEWS.csv')
mb

,Unnamed: 0,Title,Author,Date,Text
0,0,'Thank you gov': Cong Roman mourns passing of ...,Ellson Quismorio,"May 5, 2023",Bataan 1st district Rep. Geraldine Roman paid ...
1,1,P5-M reward offered to find missing Chinese pa...,Martin Sadongdong,"May 5, 2023",A P5-million reward was offered to anyone who ...
2,2,"Hontiveros: Nuclear energy a terrible, false s...",Hannah Torregoza,"May 5, 2023","\nSenator Risa Hontiveros on Friday, May 5 qu..."
3,3,You go girl! Pro-sports solon hails Kaila Napo...,Ellson Quismorio,"May 5, 2023",The first one is always special.\nPuwersa ng B...
4,4,Expedite release of Excellence in Teacher Educ...,Merlina Hernando-Malipot,"May 5, 2023",To further strengthen teacher education in the...
...,...,...,...,...,...
1999,1999,"Diokno, Australian envoy share notes on deepen...",Raymund Antonio,"Mar 25, 2023",Finance Secretary Benjamin Diokno met with Aus...
2000,2000,Australian gov't drums up interest in PH airpo...,Raymund Antonio,"Mar 25, 2023",The Australian government is looking to “drum ...
2001,2001,Proposed EBET law passed on 3rd reading; here'...,Ellson Quismorio,"Mar 25, 2023",The lower chamber recently gave its final appr...
2002,2002,Immediate expansion of gov’t subsidies to kind...,Merlina Hernando-Malipot,"Mar 25, 2023",A network of advocates called for the immediat...


In [ ]:
mb_df = make_annotateDF(mb)
mb_df

,Annotate,Text
0,,'Thank you gov': Cong Roman mourns passing of ...
1,,Bataan 1st district Rep. Geraldine Roman paid ...
2,,"""It is with a heavy heart that I offer my sinc..."
3,,"""His to enriching the lives of others through..."
4,,"""Thank you Governor Padilla for your service. ..."
...,...,...
23804,,"""The Philippines and China also discussed new ..."
23805,,"On Thursday, Lazaro started chairing with Chin..."
23806,,"The conduct of the BCM, a confidence-building ..."
23807,,"In an earlier pronouncement, Lazaro asserted t..."


In [22]:
mbHL_df = get_headlineLead(mb)
mbHL_df

,Text,Annotate
0,'Thank you gov': Cong Roman mourns passing of ...,
1,Bataan 1st district Rep. Geraldine Roman paid ...,
2,P5-M reward offered to find missing Chinese pa...,
3,A P5-million reward was offered to anyone who ...,
4,"Hontiveros: Nuclear energy a terrible, false s...",
...,...,...
3868,The lower chamber recently gave its final appr...,
3869,Immediate expansion of gov’t subsidies to kind...,
3870,A network of advocates called for the immediat...,
3871,"P​H, China vow UNCLOS​ compliance, seek Indo-P...",


In [ ]:
mb_df.iloc[19]

Annotate                                                     
Text        Hontiveros reiterated her belief that nuclear ...
Name: 19, dtype: object

## END

In [ ]:
ALL_df = pd.concat([gma_df, pna_df, rappler_df, sunstar_df, mb_df])
ALL_df

# ALL_df.to_csv(f'{dir}/Official/curated_Dataset.csv', index=False)


In [23]:
ALL_HL_df = pd.concat([gmaHL_df, pnaHL_df, rapplerHL_df, sunstarHL_df, mbHL_df])
ALL_HL_df

ALL_HL_df.to_csv(f'{dir}/Official/curated_Dataset_headlines_lead.csv', index=False)

In [ ]:
ALL_HL_df

,Text,Annotate
0,"DOH logs 1,920 new COVID-19 cases, highest dai...",
1,"The Philippines on Sunday registered 1,920 new...",
2,"DOH logs 9 new COVID-19 deaths; 1,878 fresh ca...",
3,The Department of Health said Saturday that th...,
4,Bettor wins P55.3 million jackpot in Grand Lot...,
...,...,...
3868,The lower chamber recently gave its final appr...,
3869,Immediate expansion of gov’t subsidies to kind...,
3870,A network of advocates called for the immediat...,
3871,"P​H, China vow UNCLOS​ compliance, seek Indo-P...",


In [ ]:
ALL_df

,Annotate,Text
0,,"DOH logs 1,920 new COVID-19 cases, highest dai..."
1,,"The Philippines on Sunday registered 1,920 ne..."
2,,"According to GMA Integrated News Research, th..."
3,,"It is the fourth straight day of more than 1,..."
4,,Data from the Department of Health's COVID-19...
...,...,...
23804,,"""The Philippines and China also discussed new ..."
23805,,"On Thursday, Lazaro started chairing with Chin..."
23806,,"The conduct of the BCM, a confidence-building ..."
23807,,"In an earlier pronouncement, Lazaro asserted t..."
